In [2]:
from game_logic.ghostAgents import RandomGhost
from entregables.maxNAgent import MaxNAgent
from game_logic.randomPacman import RandomPacman
from game_logic.PacmanEnvAbs import PacmanEnvAbs
import random
import math
import numpy as np
from game_logic.game_util import process_state
from main import get_default_agents
import timeit 
import pandas as pd

all_layouts = [
        "custom1",
        "custom2",
        "capsuleClassic",
        "contestClassic",
        "mediumClassic",
        "minimaxClassic",
        "openClassic",
        "originalClassic",
        "smallClassic",
        "testClassic",
        "trappedClassic",
        "trickyClassic",
        "mediumGrid",
        "smallGrid"
    ]

# Parametros para un test
class TestParams():
    def __init__(self, test_name, layout, pacman_agent, ghost_agent_0, ghost_agent_1):        
        self.test_name = test_name
        self.layout = layout
        self.pacman_agent = pacman_agent
        self.ghost_agent_0 = ghost_agent_0
        self.ghost_agent_1 = ghost_agent_1


In [3]:
def run_test(test_params):  
    t0 = timeit.default_timer()   
    pacman_agent = test_params.pacman_agent
    ghost_agent_0 = test_params.ghost_agent_0
    ghost_agent_1 = test_params.ghost_agent_1
    agents = [pacman_agent, ghost_agent_0, ghost_agent_1]
    agents.extend(get_default_agents(3, 10))    
    done = False
    env = PacmanEnvAbs(agents = agents, view_distance = (2, 2))      
    game_state = env.reset(enable_render= False, layout_name= test_params.layout)
    turn_index = 0
    while (not(done)):
        action = agents[turn_index].getAction(game_state)
        game_state, rewards, done, info = env.step(action, turn_index)        
        turn_index = (turn_index + 1) % env._get_num_agents()  
    t1 = timeit.default_timer()    
    time = t1-t0  
    assert(game_state.getNumAgents()>=2)
    if game_state.getNumAgents()==2:
        ret = game_state.get_rewards()
        ret.append(np.nan)
    else:
        ret = game_state.get_rewards()[0:3]
    return ret, time

In [4]:
pacman_agent = RandomPacman(index = 0)
ghost_agent_0 = MaxNAgent(index = 1, unroll_type="MCTS", max_unroll_depth=12, number_of_unrolls=6)
ghost_agent_1 = RandomGhost(index = 2)
sample_test = TestParams("PrimerTest", "mediumGrid", pacman_agent, ghost_agent_0, ghost_agent_1)
print(run_test(sample_test))


([-1509, -4256.0, nan], 1.6934215809997113)


In [5]:
selected_layouts = [  
        "smallClassic"
    ]
# 1 pacman:
    # MCTS --> Eval 1 (get_rewards)
pacmans = [
    ("MAXN_pcmn",MaxNAgent(index = 0, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10))
]

# 6 ghosts:
    # MC ---> Eval 1 (get_rewards)
    # MC ---> Eval 2 (get_rewards)
    # MC ---> Eval 3 (get_rewards)
    # MC ---> Eval 1 (use_eval_in_end)
    # MC ---> Eval 2 (use_eval_in_end)
    # MC ---> Eval 3 (use_eval_in_end)    
ghosts0s = [
    ("Eval1_MC_gh0", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=1, use_eval_in_end=False)),
    ("Eval2_MC_gh0", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=2, use_eval_in_end=False)),
    ("Eval3_MC_gh0", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=3, use_eval_in_end=False)),
    ("Eval1_MC_gh0_isEval", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=1, use_eval_in_end=True)),
    ("Eval2_MC_gh0_isEval", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=2, use_eval_in_end=True)),
    ("Eval3_MC_gh0_isEval", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10, eval_id=3, use_eval_in_end=True))    
]

# 6 ghosts:
    # MCTS ---> Eval 1 (get_rewards)
    # MCTS ---> Eval 2 (get_rewards)
    # MCTS ---> Eval 3 (get_rewards)
    # MCTS ---> Eval 1 (use_eval_in_end)
    # MCTS ---> Eval 2 (use_eval_in_end)
    # MCTS ---> Eval 3 (use_eval_in_end) 
ghosts1s = [
    ("Eval1_MCTS_gh1", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=1, use_eval_in_end=False)),
    ("Eval2_MCTS_gh1", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=2, use_eval_in_end=False)),
    ("Eval3_MCTS_gh1", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=3, use_eval_in_end=False)),
    ("Eval1_MCTS_gh1_isEval", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=1, use_eval_in_end=True)),
    ("Eval2_MCTS_gh1_isEval", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=2, use_eval_in_end=True)),
    ("Eval3_MCTS_gh1_isEval", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10, eval_id=3, use_eval_in_end=True))    
]

In [6]:
all_tests = []
for layout in selected_layouts:
    for pacman_name, pacman_agent in pacmans:
        for ghost0_name, ghost_0 in ghosts0s:
            for ghost1_name, ghost_1 in ghosts1s:
                # Sacamos el layout y pacman porque usamos siempre smallClassic y MAXN_pcmn (MCTS) en todos los test.
                name = ghost0_name+"_"+ghost1_name
                test = TestParams(name, layout, pacman_agent, ghost_agent_0, ghost_agent_1)
                all_tests.append(test)
number_of_runs = 10

In [7]:
results = []
t0 = timeit.default_timer()   
for i in range(len(all_tests)):   
        values = []
        times = []    
        for _ in range(number_of_runs):
            value, time = run_test(all_tests[i])            
            values.append(value)
            times.append(time)
        value_avg = np.round(np.mean(np.array(values), axis=0),2)
        time_avg = np.round(np.mean(np.array(times)),2)
        value_std = np.round(np.std(np.array(values), axis=0),2)
        time_std = np.round(np.std(np.array(times)),2)
        result = {'TestName':all_tests[i].test_name,'Time_Avg':time_avg,'Time_Std':time_std, 
                  'P_Value_Avg':value_avg[0], 
                  'P_Value_Std':value_std[0],
                  'GH0_Value_Avg':value_avg[1], 
                  'GH0_Value_Std':value_std[1],
                  'GH1_Value_Avg':value_avg[2], 
                  'GH1_Value_Std':value_std[2]
                  }
        results.append(result)
results = pd.DataFrame(results)
t1 = timeit.default_timer()    
time = t1-t0  
print("All tests time: ", time, "seconds.")  
pd.set_option('display.max_rows', 100)  
display(results)

All tests time:  9485.123584251 seconds.


,TestName,Time_Avg,Time_Std,P_Value_Avg,P_Value_Std,GH0_Value_Avg,GH0_Value_Std,GH1_Value_Avg,GH1_Value_Std
0,Eval1_MC_gh0_Eval1_MCTS_gh1,27.20,15.26,-1406.1,171.32,-26501.1,14846.20,-29847.3,17336.20
1,Eval1_MC_gh0_Eval2_MCTS_gh1,37.09,28.88,-1319.2,298.55,-36418.0,28689.92,-41386.3,32917.81
2,Eval1_MC_gh0_Eval3_MCTS_gh1,21.49,18.49,-1399.3,159.22,-20977.8,18174.14,-23669.3,21031.45
3,Eval1_MC_gh0_Eval1_MCTS_gh1_isEval,26.20,14.44,-1466.8,130.67,-25707.3,14201.17,-28962.0,16539.82
4,Eval1_MC_gh0_Eval2_MCTS_gh1_isEval,21.24,9.79,-1367.3,183.44,-20817.0,9460.02,-23187.2,11091.58
5,Eval1_MC_gh0_Eval3_MCTS_gh1_isEval,15.20,15.54,-1466.7,79.84,-14705.7,14908.15,-16342.4,17751.03
6,Eval2_MC_gh0_Eval1_MCTS_gh1,14.82,16.21,-1414.8,126.44,-14339.8,15769.97,-16028.4,18408.28
7,Eval2_MC_gh0_Eval2_MCTS_gh1,20.36,12.11,-1403.8,184.11,-19950.7,11855.68,-22174.7,13813.43
8,Eval2_MC_gh0_Eval3_MCTS_gh1,28.89,25.88,-1367.1,211.64,-28290.2,25298.10,-31826.1,29377.39
9,Eval2_MC_gh0_Eval1_MCTS_gh1_isEval,27.15,12.69,-1461.2,145.54,-26493.1,12350.39,-30000.5,14491.41
